## Importar librerías y configuración

In [1]:
import boto3
import time

glue = boto3.client('glue', region_name='us-east-1')  # Cliente de Glue
s3_bucket = "retail-multisource-pipeline"
processed_prefix = "processed/"
database_name = "retail_processed"
crawler_name = "crawler_retail_processed"
iam_role = "arn:aws:iam::905936428907:role/service-role/AWSGlueServiceRole-retail-S3"  # Rol con permisos


## Crear la base de datos si no existe

In [2]:
try:
    glue.create_database(DatabaseInput={"Name": database_name})
    print(f"Base de datos '{database_name}' creada.")
except glue.exceptions.AlreadyExistsException:
    print(f"Base de datos '{database_name}' ya existe.")


Base de datos 'retail_processed' ya existe.


## Crear o actualizar el Crawler

In [3]:
try:
    glue.create_crawler(
        Name=crawler_name,
        Role=iam_role,
        DatabaseName=database_name,
        Targets={
            'S3Targets': [{'Path': f"s3://{s3_bucket}/{processed_prefix}"}]
        },
        TablePrefix="",
        SchemaChangePolicy={
            'UpdateBehavior': 'UPDATE_IN_DATABASE',
            'DeleteBehavior': 'DEPRECATE_IN_DATABASE'
        }
    )
    print(f"Crawler '{crawler_name}' creado.")
except glue.exceptions.AlreadyExistsException:
    print(f"Crawler '{crawler_name}' ya existe, se actualizará la configuración.")
    glue.update_crawler(
        Name=crawler_name,
        Role=iam_role,
        DatabaseName=database_name,
        Targets={
            'S3Targets': [{'Path': f"s3://{s3_bucket}/{processed_prefix}"}]
        },
        TablePrefix="",
        SchemaChangePolicy={
            'UpdateBehavior': 'UPDATE_IN_DATABASE',
            'DeleteBehavior': 'DEPRECATE_IN_DATABASE'
        }
    )

Crawler 'crawler_retail_processed' creado.


## Ejecutar el crawler

In [4]:
glue.start_crawler(Name=crawler_name)
print("Crawler ejecutándose...")

Crawler ejecutándose...


## Esperar a que termine el crawler

In [5]:
while True:
    status = glue.get_crawler(Name=crawler_name)['Crawler']['State']
    if status == 'READY':
        print("Crawler ha terminado. Tablas listas en Glue/Athena.")
        break
    else:
        print("Crawler en ejecución, esperando 10s...")
        time.sleep(10)


Crawler en ejecución, esperando 10s...
Crawler en ejecución, esperando 10s...
Crawler en ejecución, esperando 10s...
Crawler en ejecución, esperando 10s...
Crawler en ejecución, esperando 10s...
Crawler en ejecución, esperando 10s...
Crawler en ejecución, esperando 10s...
Crawler en ejecución, esperando 10s...
Crawler en ejecución, esperando 10s...
Crawler en ejecución, esperando 10s...
Crawler en ejecución, esperando 10s...
Crawler en ejecución, esperando 10s...
Crawler ha terminado. Tablas listas en Glue/Athena.
